In [1]:
import tensorflow as tf
import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import dqn, ppo

from env import *
from model import *
from action_distribution import *

In [2]:
%load_ext autoreload
%autoreload 2

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



True

In [3]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-03-11 22:43:08,973	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.3.0


In [4]:
n_disks = 4
env_config = {
    'n_disks': n_disks,
    'max_n_steps': 10000 #2 ** (n_disks + 2)
}

def create_env(config):
    _env = HanoiTower(config)
    _env = PreprocessorEnv(_env)
    return _env

ENV_NAME = 'hanoi_tower'
register_env(ENV_NAME, create_env)

config = ppo.PPOConfig()
config = config.training(lr=1e-3,
                         gamma=0.9,
                         train_batch_size=512 * 2,
                         sgd_minibatch_size=256 * 1,
                         num_sgd_iter=16 * 1)
config = config.rollouts(num_rollout_workers=8,
                         num_envs_per_worker=4)
config = config.environment(ENV_NAME, 
                            env_config=env_config)
config = config.resources(num_gpus=1)
config = config.framework('tf')

model_config = {
    'n_hiddens': 16
}

config.model.update({
    'custom_model': MODEL_NAME,
    'custom_model_config': model_config,
    'custom_action_dist': ACTION_DIST_NAME,
    'vf_share_layers': True
})

config.seed = 19
agent = config.build()

2023-03-11 22:43:09,718	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-11 22:43:09,736	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=29593) Metal device set to: Apple M1
(RolloutWorker pid=29593) 
(RolloutWorker pid=29593) systemMemory: 16.00 GB
(RolloutWorker pid=29593) maxCacheSize: 5.33 GB
(RolloutWorker pid=29593) 
(RolloutWorker pid=29591) Metal device set to: Apple M1
(RolloutWorker pid=29591) 
(RolloutWorker pid=29591) systemMemory: 16.00 GB
(RolloutWorker pid=29591) maxCacheSize: 5.33 GB
(RolloutWorker pid=29591) 
(RolloutWorker pid=29591) Model: "base_model"
(RolloutWorker pid=29591) _________________________________________________________________
(RolloutWorker pid=29591)  Layer (type)                Output Shape              Param #   
(RolloutWorker pid=29591) =================================================================
(RolloutWorker pid=29591)  obs_input (InputLayer)      [(None, 3, 4)]            0         
(RolloutWorker pid=29591)                                                                  
(RolloutWorker pid=29591)  flatten (Flatten)           (None, 12)               

(RolloutWorker pid=29593) 2023-03-11 22:43:14,020	ERROR worker.py:772 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=29593, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x3049ef580>)
(RolloutWorker pid=29593)   File "/Users/abdulhakeemabdulrahman/miniforge3/envs/hanoi_tower/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 737, in __init__
(RolloutWorker pid=29593)     self._build_policy_map(policy_dict=self.policy_dict)
(RolloutWorker pid=29593)   File "/Users/abdulhakeemabdulrahman/miniforge3/envs/hanoi_tower/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1984, in _build_policy_map
(RolloutWorker pid=29593)     new_policy = create_policy_for_framework(
(RolloutWorker pid=29593)   File "/Users/abdulhakeemabdulrahman/miniforge3/envs/hanoi_tower/lib/python3.10/site-packages/ray/rllib/utils/policy.py", line 130, 

(RolloutWorker pid=29590) Metal device set to: Apple M1
(RolloutWorker pid=29590) 
(RolloutWorker pid=29590) systemMemory: 16.00 GB
(RolloutWorker pid=29590) maxCacheSize: 5.33 GB
(RolloutWorker pid=29590) 
(RolloutWorker pid=29590) Model: "base_model"
(RolloutWorker pid=29590) _________________________________________________________________
(RolloutWorker pid=29590)  Layer (type)                Output Shape              Param #   
(RolloutWorker pid=29590) =================================================================
(RolloutWorker pid=29590)  obs_input (InputLayer)      [(None, 3, 4)]            0         
(RolloutWorker pid=29590)                                                                  
(RolloutWorker pid=29590)  flatten (Flatten)           (None, 12)                0         
(RolloutWorker pid=29590)                                                                  
(RolloutWorker pid=29590)  context (Dense)             (None, 16)                208       
(RolloutWor

(RolloutWorker pid=29590) 2023-03-11 22:43:14,197	ERROR worker.py:772 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=29590, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x16c1eb670>)
(RolloutWorker pid=29590)   File "/Users/abdulhakeemabdulrahman/miniforge3/envs/hanoi_tower/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 737, in __init__
(RolloutWorker pid=29590)     self._build_policy_map(policy_dict=self.policy_dict)
(RolloutWorker pid=29590)   File "/Users/abdulhakeemabdulrahman/miniforge3/envs/hanoi_tower/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1984, in _build_policy_map
(RolloutWorker pid=29590)     new_policy = create_policy_for_framework(
(RolloutWorker pid=29590)   File "/Users/abdulhakeemabdulrahman/miniforge3/envs/hanoi_tower/lib/python3.10/site-packages/ray/rllib/utils/policy.py", line 130, 

ValueError: The two structures don't have the same nested structure.

First structure: type=Tensor str=Tensor("default_policy_wk1/cond/cond/concat:0", shape=(?, 2), dtype=int64)

Second structure: type=tuple str=(<tf.Tensor 'default_policy_wk1/cond/cond/ArgMax:0' shape=(?,) dtype=int64>, <tf.Tensor 'default_policy_wk1/cond/cond/ArgMax_1:0' shape=(?,) dtype=int64>)

More specifically: Substructure "type=tuple str=(<tf.Tensor 'default_policy_wk1/cond/cond/ArgMax:0' shape=(?,) dtype=int64>, <tf.Tensor 'default_policy_wk1/cond/cond/ArgMax_1:0' shape=(?,) dtype=int64>)" is a sequence, while substructure "type=Tensor str=Tensor("default_policy_wk1/cond/cond/concat:0", shape=(?, 2), dtype=int64)" is not
Entire first structure:
.
Entire second structure:
(., .)

In [ ]:
config.to_dict()

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 1,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'num_trainer_workers': 0,
 'num_gpus_per_trainer_worker': 0,
 'num_cpus_per_trainer_worker': 1,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': 'hanoi_tower',
 'env_config': {'n_disks': 4, 'max_n_steps': 10000},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 '

In [ ]:
for _ in range(50):
    res = agent.train()

In [7]:
env = create_env(env_config)
obs, _ = env.reset()

episode_reward = 0
done = False
while not done:
    action = agent.compute_single_action(obs, 
                                         explore=False)
    print(action)
    obs, reward, done, terminated, info = env.step(action)
    episode_reward += reward


[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2]
[1 2

KeyboardInterrupt: 

In [11]:
obs

array([[0, 0, 0],
       [0, 0, 0],
       [1, 2, 3]], dtype=int32)

In [12]:
episode_reward

-7.0

In [ ]:
env.observation_space

Box(0, 2, (3, 2), int32)

In [5]:
env = HanoiTower(config=env_config)

In [6]:
env.action_space.sample()

array([2, 1])

In [ ]:
obs = env.reset()
env.observation_space.contains(obs)

In [ ]:
obs

In [ ]:
env.step([0, 1])